In [1]:
import boto3
import pandas as pd
from faker import Faker
import random
from datetime import datetime
from io import StringIO

In [ ]:
fake = Faker()
s3 = boto3.client(
    "s3",
    aws_access_key_id='', # aws access key
    aws_secret_access_key='', # aws secret access key
    region_name='' # aws region name 
)

In [15]:
def generate_cdc_order_data(num_rows = 500) :
    data = []
    for _ in range(num_rows):
        order = {
            'order_id': fake.uuid4(),
            'customer_id': fake.uuid4(),
            'order_date': fake.date_this_year(),
            'status': random.choice(['CREATED', 'SHIPPED', 'DELIVERED', 'CANCELLED']),
            'product_id': fake.uuid4(),
            'quantity': random.randint(1, 5),
            'price': round(random.uniform(10.0, 500.0), 2),
            'total_amount': 0.0,  
            'cdc_timestamp': datetime.now()   
        }
        order['total_amount'] = round(order['quantity'] * order['price'] , 2)
        data.append(order)
    #convert to Dataframe
    df = pd.DataFrame(data)
    return df

In [16]:
# Function to upload data to S3
def upload_to_s3(bucket_name, file_name, df):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())
    print(f"Date uploaded to s3://{bucket_name}/{file_name}")

# Generate num_rows rows of fake CDC order data 
df_cdc_order_data = generate_cdc_order_data(num_rows = 200)

# Define S3 bucket  and file path
bucket_name = 'datalakebucket13'
file_name = 'cdc/orders_data.csv'

# Upload the generated data to S3 
upload_to_s3(bucket_name, file_name, df_cdc_order_data)

Date uploaded to s3://datalakebucket13/cdc/orders_data.csv
